In [1]:
import re



lines = open('movie_lines.txt', encoding='utf-8',
             errors='ignore').read().split('\n')

convers = open('movie_conversations.txt', encoding='utf-8',
             errors='ignore').read().split('\n')


exchn = []
for conver in convers:
    exchn.append(conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())

diag = {}
for line in lines:
    diag[line.split(' +++$+++ ')[0]] = line.split(' +++$+++ ')[-1]

## delete
del(lines, convers, conver, line)

questions = []
answers = []

for conver in exchn:
    for i in range(len(conver) - 1):
        questions.append(diag[conver[i]])
        answers.append(diag[conver[i+1]])

## delete
del(diag, exchn, conver, i)


###############################
#        max_len = 13         #
###############################

sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 13:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])



def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

clean_ques = []
clean_ans = []

for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))



## delete
del(answers, questions, line)



for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:11])



###############################
#                             #
###############################

del(sorted_ans, sorted_ques)


## trimming
clean_ans=clean_ans[:30000]
clean_ques=clean_ques[:30000]
## delete


###  count occurences ###
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

## delete
del(word, line)


###  remove less frequent ###
thresh = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1
        
## delete
del(word2count, word, count, thresh)       
del(word_num)        



for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1
    
    

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

## delete
del(token, tokens) 
del(x)

### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}



## delete
del(i)



encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

### delete
del(clean_ans, clean_ques, line, lst, word)






from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 13, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 13, padding='post', truncating='post')




decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 13, padding='post', truncating='post')


del(i)

from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))



print(decoder_final_output.shape)


(30000, 13, 3027)


In [2]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)


2.12.0
2.12.0


In [141]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('my_model.h5')
print("Model loaded successfully.")

# Continue with training or further operations on the loaded model
# model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=30)


Model loaded successfully.


In [3]:


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input


enc_inp = Input(shape=(13, ))
dec_inp = Input(shape=(13, ))


VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
                  input_length=13,
                  trainable=True                  
                  )


enc_embed = embed(enc_inp)
enc_lstm = LSTM(400, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]



dec_embed = embed(dec_inp)
dec_lstm = LSTM(400, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)




model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('my_model.h5')
print("Model loaded successfully.")

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=20)

# Assuming `model` is your trained model
model.save('my_model.h5')
print("Model saved successfully.")

Epoch 1/20
938/938 [==============================] - 152s 157ms/step - loss: 3.1051 - acc: 0.4932
Epoch 2/20
938/938 [==============================] - 149s 159ms/step - loss: 2.7421 - acc: 0.5312
Epoch 3/20
938/938 [==============================] - 155s 165ms/step - loss: 2.6092 - acc: 0.5425
Epoch 4/20
938/938 [==============================] - 153s 163ms/step - loss: 2.5355 - acc: 0.5470
Epoch 5/20
938/938 [==============================] - 153s 163ms/step - loss: 2.4771 - acc: 0.5506
Epoch 6/20
938/938 [==============================] - 153s 163ms/step - loss: 2.4267 - acc: 0.5534
Epoch 7/20
938/938 [==============================] - 152s 163ms/step - loss: 2.3813 - acc: 0.5557
Epoch 8/20
938/938 [==============================] - 153s 163ms/step - loss: 2.3372 - acc: 0.5581
Epoch 9/20
938/938 [==============================] - 154s 164ms/step - loss: 2.2918 - acc: 0.5603
Epoch 10/20
938/938 [==============================] - 150s 159ms/step - loss: 2.2484 - acc: 0.5627
Epoch 11/

In [4]:
model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=30)
# Assuming `model` is your trained model
model.save('my_model.h5')
print("Model saved successfully.")

Epoch 1/30
938/938 [==============================] - 156s 166ms/step - loss: 1.8262 - acc: 0.6037
Epoch 2/30
938/938 [==============================] - 156s 167ms/step - loss: 1.7927 - acc: 0.6091
Epoch 3/30
938/938 [==============================] - 161s 171ms/step - loss: 1.7610 - acc: 0.6138
Epoch 4/30
938/938 [==============================] - 156s 167ms/step - loss: 1.7283 - acc: 0.6197
Epoch 5/30
938/938 [==============================] - 155s 165ms/step - loss: 1.6981 - acc: 0.6245
Epoch 6/30
938/938 [==============================] - 156s 167ms/step - loss: 1.6681 - acc: 0.6295
Epoch 7/30
938/938 [==============================] - 156s 166ms/step - loss: 1.6390 - acc: 0.6353
Epoch 8/30
938/938 [==============================] - 156s 166ms/step - loss: 1.6100 - acc: 0.6403
Epoch 9/30
938/938 [==============================] - 156s 166ms/step - loss: 1.5825 - acc: 0.6455
Epoch 10/30
938/938 [==============================] - 159s 169ms/step - loss: 1.5548 - acc: 0.6506
Epoch 11/

In [5]:
model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=30)
# Assuming `model` is your trained model
model.save('my_model.h5')
print("Model saved successfully.")

Epoch 1/30
938/938 [==============================] - 154s 164ms/step - loss: 1.0864 - acc: 0.7510
Epoch 2/30
938/938 [==============================] - 152s 162ms/step - loss: 1.0701 - acc: 0.7543
Epoch 3/30
938/938 [==============================] - 150s 160ms/step - loss: 1.0521 - acc: 0.7585
Epoch 4/30
938/938 [==============================] - 152s 162ms/step - loss: 1.0357 - acc: 0.7622
Epoch 5/30
938/938 [==============================] - 153s 163ms/step - loss: 1.0194 - acc: 0.7663
Epoch 6/30
938/938 [==============================] - 153s 163ms/step - loss: 1.0032 - acc: 0.7701
Epoch 7/30
938/938 [==============================] - 151s 161ms/step - loss: 0.9885 - acc: 0.7732
Epoch 8/30
938/938 [==============================] - 149s 159ms/step - loss: 0.9724 - acc: 0.7771
Epoch 9/30
938/938 [==============================] - 149s 159ms/step - loss: 0.9586 - acc: 0.7801
Epoch 10/30
938/938 [==============================] - 151s 161ms/step - loss: 0.9441 - acc: 0.7838
Epoch 11/

In [142]:
# Assuming `model` is your trained model
model.save('my_model.h5')
print("Model saved successfully.")

Model saved successfully.


In [144]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('my_model.h5')
print("Model loaded successfully.")

# Continue with training or further operations on the loaded model
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=1)
# Assuming `model` is your trained model
model.save('my_model.h5')
print("Model saved successfully.")


Model loaded successfully.
938/938 [==============================] - 171s 176ms/step - loss: 0.7199 - acc: 0.8366
Model saved successfully.


In [145]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


enc_model = Model([enc_inp], enc_states)



# decoder Model
decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)




In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spellchecker import SpellChecker

from IPython.display import display, HTML, clear_output

chat_history = []
spell = SpellChecker()

def print_chat(message, is_user=False):
    user_name = "Your Name"
    chatbot_name = "Chatbot"
    user_icon = "social-media.gif"
    chatbot_icon = "physics.gif"

    message_class = "user-message" if is_user else "chatbot-message"
    message_tag = user_name if is_user else chatbot_name
    message_icon = user_icon if is_user else chatbot_icon
    message_style = "background-color: #9cd6d5; text-align: left;" if is_user else "background-color: #E0E0E0; text-align: right;"
    message_align = "left" if is_user else "right"

    if message:
        if is_user:
            message_html = f'''
                <div class="message {message_class}" style="margin: 10px 0; padding: 10px; border-radius: 5px; {message_style}">
                    <div style="display: flex; align-items: center; justify-content: {message_align};">
                        <img src="{message_icon}" alt="Icon" style="width: 30px; height: 30px; margin-right: 10px;">
                        <div>
                            <span class="message-name" style="font-weight: bold; color: #1E88E5;">{message_tag}</span>
                            <span class="message-text">{message}</span>
                        </div>
                    </div>
                </div>
            '''
        else:
            message_html = f'''
                <div class="message {message_class}" style="margin: 10px 0; padding: 10px; border-radius: 5px; {message_style}">
                    <div style="display: flex; align-items: center; justify-content: {message_align};">
                        <div>
                            <span class="message-text">{message}</span>
                            <span class="message-name" style="font-weight: bold; color: #1E88E5; margin-{message_align}: 10px;">{message_tag}</span>
                        </div>
                        <img src="{message_icon}" alt="Icon" style="width: 30px; height: 30px; margin-{message_align}: 10px;">
                    </div>
                </div>
            '''
    else:
        message_html = f'''
            <div class="message chatbot-message" style="margin: 10px 0; padding: 10px; border-radius: 5px; {message_style}">
                <div style="display: flex; align-items: center; justify-content: {message_align};">
                    <div>
                        <span class="message-text">{message}</span>
                        <span class="message-name" style="font-weight: bold; color: #1E88E5; margin-{message_align}: 10px;">{message_tag}</span>
                    </div>
                    <img src="{message_icon}" alt="Icon" style="width: 30px; height: 30px; margin-{message_align}: 10px;">
                </div>
            </div>
        '''

    chat_history.append(message_html)
    display(HTML(message_html))
    clear_output(wait=True)
    display(HTML(''.join(chat_history)))


print("##########################################")
print("#       Start Chatting ver. 1.0          #")
print("##########################################")

prepro1 = ""
prev_user_input = ""
while True:
    prepro1 = input("You: ")
    prepro1 = clean_text(prepro1)

    # Correct user input with typos
    words = prepro1.split()
    corrected_words = []
    for word in words:
        corrected_word = spell.correction(word)
        corrected_words.append(corrected_word)
    prepro1 = ' '.join(corrected_words)

    prepro = [prepro1]

    txt = []
    for x in prepro:
        lst = []
        for y in x.split():
            try:
                lst.append(vocab[y])
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    txt = pad_sequences(txt, 13, padding='post')

    stat = enc_model.predict(txt, verbose=0)

    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = vocab['<SOS>']

    stop_condition = False
    decoded_translation = ''
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq] + stat, verbose=0)
        decoder_concat_input = dense(dec_outputs)

        sampled_word_index = np.argmax(decoder_concat_input[0, -1, :])
        sampled_word = inv_vocab[sampled_word_index] + ' '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
            stop_condition = True

        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index
        stat = [h, c]

    if prepro1 != prev_user_input:
        print_chat(prepro1, is_user=True)
        prev_user_input = prepro1

    if decoded_translation:
        if "<OUT>" in decoded_translation:
            print_chat("I'm sorry, I don't have an answer for that.", is_user=False)
        else:
            # Check for grammatical correction
            corrected_translation = ''
            tokens = nltk.word_tokenize(decoded_translation)
            tagged_tokens = nltk.pos_tag(tokens)
            for word, tag in tagged_tokens:
                if tag.startswith('V'):  # Verb
                    synsets = wordnet.synsets(word, pos=wordnet.VERB)
                    if synsets:
                        corrected_translation += synsets[0].lemmas()[0].name().replace('_', ' ') + ' '
                    else:
                        corrected_translation += word + ' '
                else:
                    corrected_translation += word + ' '

            print_chat(corrected_translation, is_user=False)
    else:
        print_chat("I'm sorry, I don't have an answer for that.", is_user=False)

# in this code just make the user keyword input area look better, only that no , more please